## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0

    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [28]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        input_shape = 28*28  # 输入形状为28x28的图像
        hidden_units = 2048      # 隐藏层单元数
        num_classes = 10        # 类别数

        self.W1 = tf.Variable(shape=[input_shape, hidden_units], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[input_shape, hidden_units],
                                                         minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[hidden_units], dtype=tf.float32, initial_value=tf.zeros(hidden_units))
        self.W2 = tf.Variable(shape=[hidden_units, num_classes], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[hidden_units, num_classes],
                                                         minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[num_classes], dtype=tf.float32, initial_value=tf.zeros(num_classes))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]

        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        ####################
        return logits

model = myModel() # 模型

optimizer = optimizers.Adam() # 优化器

## 计算 loss

In [29]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [30]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer,
                                    tf.constant(train_data[0], dtype=tf.float32),
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model,
                      tf.constant(test_data[0], dtype=tf.float32),
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.639688 ; accuracy 0.15216666
epoch 1 : loss 2.4243796 ; accuracy 0.19223334
epoch 2 : loss 2.2714403 ; accuracy 0.22615
epoch 3 : loss 2.1501884 ; accuracy 0.25946668
epoch 4 : loss 2.050488 ; accuracy 0.2931
epoch 5 : loss 1.9665585 ; accuracy 0.32428333
epoch 6 : loss 1.8941673 ; accuracy 0.35475
epoch 7 : loss 1.8301498 ; accuracy 0.38651666
epoch 8 : loss 1.7723024 ; accuracy 0.414
epoch 9 : loss 1.7191906 ; accuracy 0.4398
epoch 10 : loss 1.6699094 ; accuracy 0.463
epoch 11 : loss 1.6238807 ; accuracy 0.48471665
epoch 12 : loss 1.5807171 ; accuracy 0.5043333
epoch 13 : loss 1.5401382 ; accuracy 0.5219667
epoch 14 : loss 1.5019252 ; accuracy 0.5381
epoch 15 : loss 1.4658947 ; accuracy 0.55215
epoch 16 : loss 1.4318862 ; accuracy 0.56561667
epoch 17 : loss 1.3997557 ; accuracy 0.57778335
epoch 18 : loss 1.3693718 ; accuracy 0.58923334
epoch 19 : loss 1.340613 ; accuracy 0.60031664
epoch 20 : loss 1.3133682 ; accuracy 0.6109167
epoch 21 : loss 1.2875339 ; accuracy 0.